<a href="https://colab.research.google.com/github/momo54/minfo2022/blob/main/MasterInfo_UE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# grrr

Bon...
L'extraction excel de pdf ne peut être miraculeux..

Quand un tableau est à cheval sur plusieurs page, le tableau est mal reconstitué...

Il faut avoir les tableaux d'origine !!

Donc un read-Excel ne pourra donner de bons résultats...

In [1]:
!pip install pyspark
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 57 kB/s 
     |████████████████████████████████| 199 kB 68.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=27c7987c8f602d0fe9582e971299ca93337eda6b70377ce08e3f7473d5e3abad
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [3]:
import pandas as pd

##
## Autre Approche
## Même si 1 sheet != 1 UE
## l'odre est préservé
## regexp extraction en suivant l'ordre...

my_csv=r'/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'

df=pd.read_excel(my_csv, index_col=None, header=None, sheet_name="Table 5")
#Title and code of UE not at the right Place :(
# need swapping...
#print(df)
#print(df.dtypes)
mask=df.apply(lambda x: x.str.contains('X1.*|Objectifs|Contenu',regex=True)).any(axis=1)
print(df[mask])



                                        0  \
1                                 X1II010   
14  Objectifs (résultats d'apprentissage)   
15                                Contenu   
19                                X1II030   
32  Objectifs (résultats d'apprentissage)   
33                                Contenu   

                                                    1  
1                                             Graphes  
14  1. reconnaître et modéliser un problème réel a...  
15  •\nEléments de théorie des graphes\n•\nArbres ...  
19                          Complexité et algorithmes  
32  • Savoir manipuler les notations standards des...  
33                                                NaN  


In [57]:
#
# DS M2 et Vico M2 n'ont pas les code UE pour les UE...
# added it by hand in excel files.... -> withcode.xlsx
#

parcours=[
    ('ALMA','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'),
    ('ALMA','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Architectures_Logicielles_(ALMA)_2022-2023.xlsx'),
    ('ATAL','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023 (2).xlsx'),
    ('ATAL','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023.xlsx'),
    ('ORO','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Optimisation_en_Recherche_Operationnelle_(ORO)_2022-2023.xlsx'),
    ('ORO','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Optimisation_en_Recherche_Operationnelle_(ORO)_2022-2023.xlsx'),
    ('DS','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Data_Science_(DS)_2022-2023.xlsx'),
    ('DS','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Data_Science_(DS)_2022-2023-withcode.xlsx'),
    ('VICO','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Visual_Computing_(VICO)_2022-2023.xlsx'),
    ('VICO','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Visual_Computing_(VICO)-withcode.xlsx'),
    ('CMI','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_CMI-OPTIM_2022-2023.xlsx'),
    ('CMI','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_CMI-OPTIM_2022-2023.xlsx')
]


def extract_ue(parcours,an,path):
  print(f"extracting ues for {parcours}, annee:{an}, path:{path}")
  dfs=[]
#  my_csv=r'/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'

  for sheet_name, df in pd.read_excel(path, index_col=None, header=None, sheet_name=None).items():
      sheetid=int(sheet_name.split(" ")[1])
      if sheetid>4:
        print(sheet_name)
        dfs.append(df)
  allUE=pd.concat(dfs).reset_index(drop=True)

  mask=allUE.apply(lambda x: x.str.contains('^X[1-4].*|^ZZ|^YG.*|Object|Contenu',regex=True)).any(axis=1)
  UES=allUE[mask]

  #debug
  #print(UES.columns.values)
  #print(UES)
  #i don't know why i need 3 colmns sometimes...
  # read_excel pb ???
  if len(UES.columns.values)==2:
    UES.set_axis(['X','Y'],axis=1,inplace=True)
  else:
    UES.set_axis(['X','Y','Z'],axis=1,inplace=True)
    
  #debug
  #print(UES.columns.values)
  #print(UES)
  #for index, row in UES.iterrows():
  #  print(f"extracting:{row['X']}")

  state=0
  key=""
  title=""
  objective=""
  content=""
  lines=[]

  import re
  reg=re.compile("^X[1-4].*|^ZZ|^YG.*")

  for index, row in UES.iterrows():
    #print(f"extracting:{row['X']}")
    #print(row['X'],row['Y']
    if state==0:
      if reg.match(str(row['X'])) is not None:
        key=row['X']
        title=row['Y']
        state=1
      else:
        print(f"error: should read an UE key:{row['X']}")
        state=0
    elif state==1:
      if str(row['X']).startswith("Object"):
        objective=row['Y']
        state=2
      else:
        print(f"error: should read an Objective:{row['X']}")
        state=0
    elif state==2:
      if str(row['X']).startswith("Contenu"):
        content=row['Y']
        lines.append({'key' : key, 'title':title,'objective':objective,'content':content,'parc':parcours,'an':an,'from':path})
        state=0
      else:
        print(f"error: should read an Content:{row['X']}")
        state=0
        
  out=pd.DataFrame(lines)
  out.to_csv(f"{parcours}_{an}_UEs.csv",sep='|',index=False,header=True)
  return out

dfs=[]
for p,a,path in parcours:
  df=extract_ue(p,a,path)
  dfs.append(df)

all=pd.concat(dfs)
print(all)


extracting ues for ALMA, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx
Table 5
Table 6
Table 7
Table 8
Table 9
Table 10
Table 11
Table 12
Table 13
Table 14
Table 15
Table 16
Table 17
Table 18
Table 19
Table 20
Table 21
Table 22
Table 23
Table 24
extracting ues for ALMA, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Architectures_Logicielles_(ALMA)_2022-2023.xlsx
Table 5
Table 6
Table 7
Table 8
Table 9
Table 10
Table 11
extracting ues for ATAL, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023 (2).xlsx
Table 5
Table 6
Table 7
Table 8
Table 9
Table 10
Table 11
Table 12
Table 13
Table 14
Table 15
Table 16
Table 17
Table 18
Table 19
Table 20
Table 21
Table 22
Table 23
Table 24
extracting ues for ATAL, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Apprentissage_et_Traitement_Automatique_de_la_Langue_(A

In [ ]:
# OK CSV shoud be generated now
# Read UE in SPARK to groupby UE CODE...
# Troubles when reading CSV in spark... Some end of fields not well detected...
# Walk around on next block of code... (NO CSV)

from pyspark.sql.types import *
schema = StructType([
    StructField("key",StringType(),True), 
    StructField("title", StringType(), True),
    StructField("objective", StringType(), True),
    StructField("content", StringType(), True),
    StructField("parc", StringType(), True),
    StructField("an", StringType(), True),
    StructField("path", StringType(), True),
    ])

file_list=[]
for pa,an,path in parcours:
  file_list.append(f"/content/{pa}_{an}_UEs.csv")

#print(','.join(file_list))

data = spark.read \
    .format("csv")   \
    .option("delimiter","|") \
    .option("multiLine", True)\
    .option("header",True) \
    .schema(schema) \
    .load(file_list)
print(data.count())
data.show(209)

data.createOrReplaceTempView(f"UES")

UES_table=spark.sql("select key,title,objective, collect_set(parc), collect_set(an) from UES  group by key,title,objective,content")

UES_table.createOrReplaceTempView(f"UES_T")
#don't understand why pb in CSV...
UES_table.show(5)





In [83]:
## Work on UE data with SPARK now...

mydata=spark.createDataFrame(all.astype(str))
#mydata.show(5)

mydata.createOrReplaceTempView(f"UES")

ues=spark.sql("select key,title,objective, content, concat_ws(',',collect_set(parc)) as parcours, concat_ws(',',collect_set(an)) as an from UES  group by key,title,objective,content")
ues.show(5) 

ues.createOrReplaceTempView(f"UES_T")

!rm -rf ues-uniq.csv

ues.toPandas().to_csv('ues-uniq.csv')
#ues.write.options(header='True',delimiter='|') \
#  .csv("ues-uniq.csv")


+-------+--------------------+--------------------+--------------------+--------------------+---+
|    key|               title|           objective|             content|            parcours| an|
+-------+--------------------+--------------------+--------------------+--------------------+---+
|X1I2020|Graphes II et Rés...|1. reconnaître, m...|•\nProblème du fl...|ATAL,ORO,DS,CMI,VICO| M1|
|X1II040|Anglais scientifique|Au terme de cette...|Anglais de spécia...|ATAL,ORO,DS,CMI,A...| M1|
|X2II020|         Data Mining|A l'issue de ce c...|Le Data Mining es...|ATAL,ORO,DS,CMI,A...| M1|
|X1I1020|       Test logiciel|• To be able to c...|• Functional and ...|                ALMA| M1|
|X1LI010|Management à Visé...|A l’issue de cet ...|Autour d’une form...|   ATAL,DS,ALMA,VICO| M1|
+-------+--------------------+--------------------+--------------------+--------------------+---+
only showing top 5 rows



In [61]:
# ok now lets extracts ECTS and hours linked to UE from excel files

def extract_ects(parcours,an,path):
  print(f"extracting ues for {parcours}, annee:{an}, path:{path}")
  dfs=[]
#  my_csv=r'/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'

  data=pd.read_excel(path, index_col=None, header=None, sheet_name="Table 2")
  mask=data.apply(lambda x: x.str.contains('^X[1-4].*|^ZZ|^YG.*',regex=True)).any(axis=1)
  ects=data[mask]
  #print(ects)
  return ects

dfs=[]
for p,a,path in parcours:
  df=extract_ects(p,a,path)
  dfs.append(df)

ects=pd.concat(dfs)
print(ects)



extracting ues for ALMA, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx
extracting ues for ALMA, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Architectures_Logicielles_(ALMA)_2022-2023.xlsx
extracting ues for ATAL, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023 (2).xlsx
extracting ues for ATAL, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023.xlsx
extracting ues for ORO, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Optimisation_en_Recherche_Operationnelle_(ORO)_2022-2023.xlsx
extracting ues for ORO, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Optimisation_en_Recherche_Operationnelle_(ORO)_2022-2023.xlsx
extracting ues for DS, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converte

In [63]:
## Work with ECTS with SPARK

#from pyspark.sql.types import *
#ects_schema = StructType([
#    StructField("0", StringType(), True)\
#    ,StructField("1", StringType(), True)
#    ,StructField("2", LongType(), True)
#    ,StructField("3", StringType(), True)
#    ,StructField("4", StringType(), True)
#    ,StructField("5", StringType(), True)
#    ,StructField("6", StringType(), True)
#    ,StructField("8", LongType(), True)
#    ,StructField("9", StringType(), True)
#    ,StructField("10", StringType(), True)
#    ])

s_ects=spark.createDataFrame(ects.astype(str))
#s_ects=spark.createDataFrame(ects,schema=ects_schema)

s_ects=s_ects.withColumnRenamed("0","title") \
  .withColumnRenamed("1","key") \
  .withColumnRenamed("2","ects") \
  .withColumnRenamed("8","hours") \



s_ects.createOrReplaceTempView(f"ECTS")

ects_table=spark.sql("select key,  cast(max(ects) as int) as ects, cast(max(hours) as int) as hours from ECTS  group by key,ects,hours")
ects_table.createOrReplaceTempView(f"ECTS_T")
ects_table.show(5)

ects_table.toPandas().to_csv('ects.csv')


+-------+----+-----+
|    key|ects|hours|
+-------+----+-----+
|X1CI010|   2|   22|
|X1I1010|   6|   60|
|X1I1020|   3|   24|
|X1I1030|   4|   32|
|X1I1030|   5|   32|
+-------+----+-----+
only showing top 5 rows



In [73]:
# I suppose now that UE has be aligned with ACM classif...
# Read the excel file and group yby ACM items

path='/content/drive/MyDrive/MasterInfo2022/ues-uniq.xlsx'
data=pd.read_excel(path, index_col=None, header=1)

#swtich ain spark now
s_acm=spark.createDataFrame(data.astype(str))
s_acm.show(5)

from pyspark.sql.functions import col, when,concat_ws, explode,split

s_acm=s_acm.select("key",concat_ws(",",*[when(col(c)=="x",c).alias(c) for c in s_acm.columns]).alias("types"))
s_acm=s_acm.select("key",split("types",",").alias("types_array"))
s_acm=s_acm.select("key",explode("types_array").alias("types"))
s_acm.show(5)

s_acm.createOrReplaceTempView("ACM_T")


+----------+-------+--------------------+--------------------+--------------------+--------------------+---+------------------------+-----------------------------+--------+---------------------+------------------------------+----------------------------------------------+--------+-------------------+-------------------+---------------+-----+------------------------------+----------+----------------+--------------+---------------------+---------------------------+---------------------------+--------------------------+---------------------+------------------------+--------------------+---------------------+-----------------------------------+
|Unnamed: 0|    key|               title|           objective|             content|            parcours| an|MSF (Proof, graph proba)|(AL) Algorithm and Complexity|Modeling|PL and formal methods|Computer architecture and orga|Concurrent, parallel and distributed computing|Networks|security and crypto|Database and mining|Search and opti|Logic|Proba fo

In [100]:
# Bon plotting now ??

results_pa={}
for pa,an,path in parcours:
  print(f"parcours:{pa}, année:{an}")
  query=f"""
  select ACM_T.types, 
      sum(ECTS_T.ects) as ects,
      sum(ECTS_T.hours) as hours
  from ACM_T,ECTS_T, UES_T 
  where 
    ACM_T.key=ECTS_T.key 
    and ACM_T.key=UES_T.key 
    and UES_T.parcours like '%{pa}%'
    and UES_T.an = '{an}'
  group by ACM_T.types
  """
  
  results_pa[pa]={}
  results_pa[pa][an]=spark.sql(query)
  results_pa[pa][an].show(5)

parc=set([p[0] for p in parcours])

results_p={}
for pa in parc:
  print(f"parcours:{pa}")
  query=f"""
  select ACM_T.types, 
      sum(ECTS_T.ects) as ects,
      sum(ECTS_T.hours) as hours
  from ACM_T,ECTS_T, UES_T 
  where 
    ACM_T.key=ECTS_T.key 
    and ACM_T.key=UES_T.key 
    and UES_T.parcours like '%{pa}%'
  group by ACM_T.types
  """
 
  results_p[pa]=spark.sql(query)
  results_p[pa].show(5)


parcours:ALMA, année:M1
+--------------------+----+-----+
|               types|ects|hours|
+--------------------+----+-----+
| parallel and dis...|  13|   96|
|          Concurrent|  13|   96|
|            Networks|   3|   24|
|Specialized Platf...|  20|  144|
|(AL) Algorithm an...|   7|   72|
+--------------------+----+-----+
only showing top 5 rows

parcours:ALMA, année:M2
+--------------------+----+-----+
|               types|ects|hours|
+--------------------+----+-----+
| parallel and dis...|   9|   72|
|          Concurrent|   9|   72|
|Specialized Platf...|   3|   24|
|            Modeling|   6|   48|
|             Society|  36|   69|
+--------------------+----+-----+
only showing top 5 rows

parcours:ATAL, année:M1
+--------------------+----+-----+
|               types|ects|hours|
+--------------------+----+-----+
| parallel and dis...|  10|   72|
|          Concurrent|  10|   72|
|            Networks|   3|   24|
|Image and sound p...|   1|   16|
|Specialized Platf...|  21| 

In [106]:
# Radar plot Now

import plotly.graph_objects as go
import plotly.offline as pyo
import numpy as np

def flatten(l):
    return [str(item) for sublist in l for item in sublist]

#categories=np.array(results_p['ALMA'].select("types").orderBy("types").collect())
categories=flatten(results_p['ALMA'].select("types").orderBy("types").collect())

pdata=[]
for p in parc:
  h=results_p[p].select("hours").rdd.map(lambda x: x.hours).collect()
  entry=go.Scatterpolar(r=h, theta=categories, name=p)
  pdata.append(entry)

fig_h = go.Figure(
      pdata,
      layout=go.Layout(
          title=go.layout.Title(text='Master Info Hours'),
          polar={'radialaxis': {'visible': True}},
          showlegend=True
      )
  )

#pyo.plot(fig)
fig_h.update_traces(fill='toself')
fig_h.show(renderer="colab")
